In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, cross_val_score, train_test_split 
from sklearn import preprocessing

In [ ]:
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv("heart.csv", sep=',')
X = df.drop("target",axis=1)
y = df["target"].values

X = pd.get_dummies(X, columns = ['cp','thal','slope'])

In [ ]:
standard_scaler = preprocessing.StandardScaler()

In [ ]:
X_standard = standard_scaler.fit_transform(X)

In [ ]:
svm = SVC()
scores = cross_val_score(svm, X_standard, y, cv=10)

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
space4svm = {
    'C': hp.uniform('C', 0, 20),
    'kernel': hp.choice('kernel', ['linear', 'sigmoid', 'poly', 'rbf']),
    'gamma': hp.uniform('gamma', 0, 20)
}

In [ ]:
def Objective4svm(params):
    clf = SVC(**params)
    accuracy = cross_val_score(clf, X_standard, y, cv = 10).mean()
    return {'loss': -accuracy, 'status': STATUS_OK}  


In [ ]:
trials4svm = Trials()
algo=tpe.suggest
best_result = fmin(
    fn=Objective4svm, 
    space=space4svm,
    algo=algo,
    max_evals=20,
    trials=trials4svm)

In [ ]:
space4svmWithScaling = {
    'C': hp.uniform('C', 0, 20),
    'kernel': hp.choice('kernel', ['linear', 'sigmoid', 'poly', 'rbf']),
    'gamma': hp.uniform('gamma', 0, 20),
    'scale': hp.choice('scale', [0, 1]),
    'normalize': hp.choice('normalize', [0, 1])
}

In [ ]:
def Objective4svmWithScale(params):
    X_ = X
    if 'normalize' in params:
        if params['normalize'] == 1:
            X_ = preprocessing.normalize(X_)
        del params['normalize']      
    if 'scale' in params:
        if params['scale'] == 1:
            X_ = preprocessing.scale(X_)       
        del params['scale']
    clf = SVC(**params)
    accuracy = cross_val_score(clf, X_, y, cv = 10).mean()
    return {'loss': -accuracy, 'status': STATUS_OK}  

In [ ]:
trials4svmScale = Trials()
algo=tpe.suggest
best_result = fmin(
    fn=Objective4svmWithScale, 
    space=space4svmWithScaling,
    algo=algo,
    max_evals=20,
    trials=trials4svmScale)